In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install gdown

In [3]:
!gdown --id 1paVMxtKtXs5f76P0dQTuXnmdN5q1c6Yn

/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1paVMxtKtXs5f76P0dQTuXnmdN5q1c6Yn
From (redirected): https://drive.google.com/uc?id=1paVMxtKtXs5f76P0dQTuXnmdN5q1c6Yn&confirm=t&uuid=88c26704-5a19-4efb-b9bd-38248cc5de35
To: /kaggle/working/DATA_SV_ver2.zip
100%|███████████████████████████████████████| 3.61G/3.61G [00:33<00:00, 109MB/s]


In [4]:
!unzip -q DATA_SV_ver2.zip > /dev/null 2>&1

In [5]:
TIMESTEPS_IN = 9
TIMESTEPS_OUT = 1
SAMPLES_PER_DAY = 6
KERNEL_SIZE_VALUE = 3
LEARNING_RATE = 0.01
EPOCHS = 20
BATCH_SIZE = 32

In [6]:
import os
import glob
import numpy as np
from PIL import Image

# Path to the month folder (e.g., 'path/to/04' for April)
years = [2019, 2020]
months = [4, 10]

TOTAL_TIMESTEPS = TIMESTEPS_IN + TIMESTEPS_OUT

base_folder = 'DATA_SV/Precipitation/AWS'

# Initialize lists to store X and y data
X_data = []
y_data = []

for year in years:
    for month in months:
        month_folder = os.path.join(base_folder, str(year), str(month).zfill(2))
        
        # Loop over each day's folder (30 folders for each day in the month)
        for day in sorted(os.listdir(month_folder)):
            day_folder = os.path.join(month_folder, day)

            # Collect daily images in chronological order
            daily_images = []

            for img_path in sorted(glob.glob(os.path.join(day_folder, 'AWS_*.tif'))):
                # Open the image and convert to numpy array
                img = Image.open(img_path)
                img_array = np.array(img)

                # Check image dimensions
                if img_array.shape != (90, 250):
                    raise ValueError(f"Unexpected image shape: {img_array.shape}")

                # Add a channel dimension and append to daily_images
                daily_images.append(img_array[..., np.newaxis])  # Shape: (90, 250, 1)

            # Group daily images into chunks of 12 timesteps (9 for input, 3 for output)
            for i in range(0, len(daily_images) - TOTAL_TIMESTEPS + 1, int(24 / SAMPLES_PER_DAY)):
                # Ensure there are exactly 12 images per sample
                if len(daily_images[i:i + TOTAL_TIMESTEPS]) == TOTAL_TIMESTEPS:
                    # Split into X and y parts
                    X_data.append(np.array(daily_images[i:i + TIMESTEPS_IN]))  # First 9 timesteps for X
                    y_data.append(np.array(daily_images[i + TIMESTEPS_OUT:i + TOTAL_TIMESTEPS]))  # Last 3 timesteps for y

# Convert X_data and y_data lists to numpy arrays suitable for ConvLSTM input
# X shape: (samples, 9, 90, 250, 1), y shape: (samples, 3, 90, 250, 1)
X_data = np.array(X_data)
y_data = np.array(y_data)

print("X_data shape:", X_data.shape)
print("y_data shape:", y_data.shape)

X_data shape: (464, 9, 90, 250, 1)
y_data shape: (464, 9, 90, 250, 1)


In [7]:
X_data = np.nan_to_num(X_data, neginf=0)  # Replaces -inf with 0
y_data = np.nan_to_num(y_data, neginf=0)  # Replaces -inf with 0

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.15, random_state=42)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

X_train shape: (394, 9, 90, 250, 1)
y_train shape: (394, 9, 90, 250, 1)


In [9]:
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, Conv2D, Conv3D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Input
# Build the ConvLSTM model
model = Sequential([
    Input(shape=(None, *X_train.shape[2:])),
    ConvLSTM2D(filters=32, kernel_size=(KERNEL_SIZE_VALUE, KERNEL_SIZE_VALUE), activation='relu',
               padding='same', return_sequences=True),
    BatchNormalization(),

    ConvLSTM2D(filters=32, kernel_size=(KERNEL_SIZE_VALUE, KERNEL_SIZE_VALUE), activation='relu',
               padding='same', return_sequences=True),
    BatchNormalization(),

    ConvLSTM2D(filters=32, kernel_size=(KERNEL_SIZE_VALUE, KERNEL_SIZE_VALUE), activation='relu',
               padding='same', return_sequences=True),
    BatchNormalization(),

    # Change Conv3D to Conv2D as output layer for spatial prediction
    Conv3D(filters=1, kernel_size=(3, 3, 3), activation='sigmoid', padding='same')
])

model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='mean_squared_error')
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_lstm2d (ConvLSTM2D)        │ (None, None, 90, 250,  │        38,144 │
│                                 │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, None, 90, 250,  │           128 │
│ (BatchNormalization)            │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_1 (ConvLSTM2D)      │ (None, None, 90, 250,  │        73,856 │
│                                 │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, None, 90, 250,  │           128 │
│ (BatchNormalization)            │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_2 (ConvLSTM2D)      │ (None, None, 90, 250,  │        73,856 │
│                                 │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, None, 90, 250,  │           128 │
│ (BatchNormalization)            │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d (Conv3D)                 │ (None, None, 90, 250,  │           865 │
│                                 │ 1)                     │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 187,105 (730.88 KB)

 Trainable params: 186,913 (730.13 KB)

 Non-trainable params: 192 (768.00 B)

In [10]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    filepath='ConvLSTM.keras', 
    monitor='val_loss',        
    mode='min',
    save_best_only=True, 
    verbose=1
)

In [11]:
# Train model
model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.2, callbacks=[checkpoint_callback])

Epoch 1/20


I0000 00:00:1731147549.500851     107 service.cc:145] XLA service 0x5803dc6263b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731147549.500896     107 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1731147549.500900     107 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1731147563.709469     107 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code

  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/opt/conda/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/opt/conda/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelbase.py", line 545, in di

In [ ]:
model = load_model('ConvLSTM.keras')

In [ ]:
y_test_pred = model.predict(X_test)

In [ ]:
y_test_pred.shape

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr
import numpy as np

# Flatten the arrays to calculate the metrics
y_test_flat = y_test.flatten()  # Shape: (24 * 90 * 250,)
predictions_flat = y_test_pred.flatten()  # Shape: (24 * 90 * 250,)

# Calculate MSE, MAE, RMSE, R2 Score, MAPE
mse = mean_squared_error(y_test_flat, predictions_flat)
mae = mean_absolute_error(y_test_flat, predictions_flat)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_flat, predictions_flat)

# Calculate MAPE and handle potential zero values
epsilon = 1e-10  # small value to avoid division by zero
mape = np.mean(np.abs((y_test_flat - predictions_flat) / (y_test_flat + epsilon))) * 100

# Calculate Pearson correlation coefficient
pearson_r, _ = pearsonr(y_test_flat, predictions_flat)

# Print metrics
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2):", r2)
print("Mean Absolute Percentage Error (MAPE):", mape, "%")
print("Pearson Correlation Coefficient (r):", pearson_r)

In [ ]:
import matplotlib.pyplot as plt

num_examples = 10  # Number of examples to visualize
plt.figure(figsize=(20, 5 * num_examples))

for i in range(num_examples):
    # Plot actual image
    plt.subplot(num_examples, 2, i * 2 + 1)
    plt.imshow(y_test[i][TIMESTEPS_IN - TIMESTEPS_OUT].reshape(90, 250), cmap='OrRd')  # Actual
    plt.title('Actual Rainfall Image')
    plt.axis('off')

    # Plot predicted image
    plt.subplot(num_examples, 2, i * 2 + 2)
    plt.imshow(y_test_pred[i][TIMESTEPS_IN - TIMESTEPS_OUT].reshape(90, 250), cmap='OrRd')  # Prediction
    plt.title('Predicted Rainfall Image')
    plt.axis('off')

plt.tight_layout()
plt.show()